In [1]:
import src.tecnologias as t
import src.leer_datos as ld
import src.obtener_modelo_dg_not_bat as om
import src.lectura_resultados_dg_not_bat as r
import src.correr_modelo as cm
import src.guardar_graficos_dg_not_bat as gg
import src.database as db
import sys
import pandas as pd
import numpy as np
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")
from time import time
import os
import ast
import sqlite3
import plotly.graph_objects as go

In [2]:
t.bateria(pcell_nom = 0.84, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 2) #--> Esto me da 40.32

t.bateria(pcell_nom = 0.7, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 2) #---> Esto me da 33.2    

t.bateria(pcell_nom = 0.7, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 3) #---> Esto me da 50.4 

t.bateria(pcell_nom = 0.7, vdc_sist = 48, 
          vdc_bc = 2, nbat_p = 4) #---> Esto me da 50.4 

(67.2, 96.0)

In [3]:
bd = ld.leer_datos("data/datos_microrred_horario_ano_sede_oriente.xlsx")
simulaciones = ld.leer_datos("data/simulaciones_2.xlsx")

In [4]:
for i in range(len(simulaciones)):
    os.mkdir("graficos/%s"%(str(simulaciones.loc[i,'id_simulacion'])))

In [5]:



def simular_all(i):

    parametros = ast.literal_eval(simulaciones.loc[i, 'parametros'])

    parametros_otros = ast.literal_eval(
        simulaciones.loc[i, 'parametros_otros'])

    costos = ast.literal_eval(simulaciones.loc[i, 'costos'])

    modelo = om.modelo(bd=bd,
                       parametros=parametros,
                       parametros_otros=parametros_otros,
                       costos=costos)

    resultado_modelo = cm.correr_modelo(
        model=modelo[0],
        gap=0.05,
        time_limit=3600,
        experimento=str(simulaciones.loc[i, 'id_simulacion']))

    generacion_pv = modelo[1]
    load = modelo[2]
    result = resultado_modelo[0]
    term_cond = str(resultado_modelo[1])

    parametros_control = r.resultado(load, generacion_pv, modelo[0])[0]
    tabla_resultados = r.resultado(load, generacion_pv, modelo[0])[1]

    tabla_resultados['recurso_diesel_dis'] = parametros['p_dg']
    tabla_resultados['id_simulacion'] = simulaciones.loc[i, 'id_simulacion']
    tabla_resultados['porcentaje_carga_dg'] = (
        (tabla_resultados['energia_Dg'] ) /
        tabla_resultados['recurso_diesel_dis']) * 100
    tabla_resultados['porcentaje_energia_dg'] = (
        (tabla_resultados['energia_Dg']) /
        tabla_resultados['recurso_diesel_dis']) * 100
    
    tabla_resultados['porcentaje_SOC_t'] = (
        (tabla_resultados['SOC(t)_bateria']) / parametros['p_bat']) * 100
    tabla_resultados = tabla_resultados[[
        'id_simulacion', 'energia_PV', 'energia_Dg', 'bin_diesel',
        'energia_carga_bateria', 'bin_bat_carga', 'energia_descarga_bateria',
        'bin_bat_descarga', 'energia_ENS', 'SOC(t)_bateria', 'p_bat_pv',
        'suma_pv_dg_bat_descarga', 'load', 'recurso_pv_dis',
        'recurso_diesel_dis', 'porcentaje_carga_dg', 'porcentaje_energia_dg',
         'porcentaje_SOC_t'
    ]]

    par_in = pd.DataFrame(simulaciones.loc[i, :][0:21]).reset_index()
    par_in['id'] = simulaciones.loc[i, 'id_simulacion']
    par_in.columns = ['var', 'val', 'id']
    par_in = par_in.pivot(
        index='id', columns="var", values='val').reset_index(drop=True)[[
            'id_simulacion', 'n_pv', 'n_dg', 'p_dg', 'min_dg', 'efi_dg',
            'lpsp', 'p_bat', 'cond_init_bat', 'val_aux_bateria', 'DOD',
            'n_bat', 'self_dis_coef', 'max_ciclos_descarga',
            'efficiency_inversor', 'efficiency_charging', 'C_rate', 'cost_pv',
            'cost_dg', 'cost_bat', 'cost_pens'
        ]]

    par_in['optimizacion'] = term_cond

    if term_cond == 'optimal':
        par_in['lpsp_result'] = parametros_control.loc[0, 'LPSP(%)']
        par_in['ens_result'] = round(parametros_control.loc[0, 'ENS'], 1)
        par_in['ciclos_bat_result'] = round(
            tabla_resultados['energia_descarga_bateria'].sum() /
            parametros['p_bat'], 1)
    else:
        par_in['lpsp_result'] = 0
        par_in['ens_result'] = 0
        par_in['ciclos_bat_result'] = 0

    db.guardar_bd(tabla_resultados=tabla_resultados,
                  par_in=par_in,
                  term_cond=term_cond)

    if term_cond == 'optimal':
        gg.graficos(bd=bd,
                    tabla_resultados=tabla_resultados,
                    i=simulaciones.loc[i, 'id_simulacion'],
                    parametros=parametros)

    print("simulacion", i + 1, "Terminada")

In [6]:
for i in range(1,len(simulaciones)):
    simular_all(i)

    print("\n\n simulado:",i+1)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\wrope\gurobi.lic
Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\72.txt
   Prev:   Default: 
Solver log file: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\72.txt
Changed value of parameter mipgap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 140162 rows, 105120 columns and 306599 nonzeros
Model fingerprint: 0xbcaf003c
Variable types: 78840 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6506e+07    0 1196          - 1.6506e+07      -     -    5s
     0     0 1.6729e+07    0  864          - 1.6729e+07      -     -    8s
     0     0 1.6734e+07    0  740          - 1.6734e+07      -     -    9s
     0     0 1.6735e+07    0  742          - 1.6735e+07      -     -    9s
     0     0 1.6735e+07    0  740          - 1.6735e+07      -     -    9s
     0     0 1.6761e+07    0  976          - 1.6761e+07      -     -   11s
     0     0 1.6761e+07    0  974          - 1.6761e+07      -     -   11s
     0     0 1.6761e+07    0  966          - 1.6761e+07      -     -   11s
     0     0 1.6768e+07    0  930          - 1.6768e+07      -     -   13s
     0     0 1.6769e+07    0  930          - 1.6769e+07      -     -   14s
     0     0 1.6769e+07    0  929          - 1.6769e+07      -     -   14s
     0     0 1.6770e+07

   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 140162 rows, 105120 columns and 306599 nonzeros
Model fingerprint: 0x3e4de6cb
Variable types: 78840 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [3e-05, 7e+00]
  Objective range  [1e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-04, 2e+03]
Presolve removed 83517 rows and 61385 columns
Presolve time: 0.80s
Presolved: 56645 rows, 43735 columns, 157648 nonzeros
Variable types: 27002 continuous, 16733 integer (16733 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.357894e+07, 22667 iterations, 1.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

Presolve time: 0.79s
Presolved: 56790 rows, 44220 columns, 158603 nonzeros
Variable types: 27321 continuous, 16899 integer (16899 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.463701e+07, 21530 iterations, 0.90 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.4637e+07    0 1167          - 1.4637e+07      -     -    4s
     0     0 1.4849e+07    0  849          - 1.4849e+07      -     -    7s
     0     0 1.4853e+07    0  682          - 1.4853e+07      -     -    8s
     0     0 1.4853e+07    0  679          - 1.4853e+07      -     -    8s
     0     0 1.4853e+07    0  676          - 1.4853e+07      -     -    8s
     0     0 1.4875e+07    0  899          - 1.4875e+07      -     -   10s
     0     0 1.4875e+07    0  904         

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.3591e+07    0 1085          - 1.3591e+07      -     -    4s
     0     0 1.3780e+07    0  738          - 1.3780e+07      -     -    8s
     0     0 1.3781e+07    0  664          - 1.3781e+07      -     -    8s
     0     0 1.3781e+07    0  660          - 1.3781e+07      -     -    8s
     0     0 1.3803e+07    0  824          - 1.3803e+07      -     -   10s
     0     0 1.3803e+07    0  825          - 1.3803e+07      -     -   10s
     0     0 1.3803e+07    0  821          - 1.3803e+07      -     -   11s
     0     0 1.3809e+07    0  816          - 1.3809e+07      -     -   13s
     0     0 1.3809e+07    0  821          - 1.3809e+07      -     -   13s
     0     0 1.3810e+07    0  817          - 1.3810e+07      -     -   14s
     0     0 1.3810e+07    0  820          - 1.3810e+07      -     -   16s
H    0     0                    1.386943e+07 1.3810e+07  0.43%     -   17s

Cutting planes:
  Gom

...

Model fingerprint: 0x20510cc1
Variable types: 78840 continuous, 26280 integer (26280 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+01]
  Objective range  [1e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-04, 2e+03]
Presolve removed 85559 rows and 62324 columns
Presolve time: 0.80s
Presolved: 54603 rows, 42796 columns, 153541 nonzeros
Variable types: 26141 continuous, 16655 integer (16655 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 8.277717e+06, 18581 iterations, 0.91 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8277716.92    0  530          - 8277716.92      -     -    3s
H    0     0                    8866290.5432 8277716.92  6.64%     -    3s
H    0     0                    8606313.158

Presolve removed 85401 rows and 62108 columns
Presolve time: 0.81s
Presolved: 54761 rows, 43012 columns, 154146 nonzeros
Variable types: 26288 continuous, 16724 integer (16724 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 9.157830e+06, 16994 iterations, 0.73 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9157830.44    0  636          - 9157830.44      -     -    3s
H    0     0                    9835693.6140 9157830.44  6.89%     -    3s
H    0     0                    9523686.3755 9157830.44  3.84%     -    3s

Explored 1 nodes (20922 simplex iterations) in 3.87 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 9.52369e+06 9.83569e+06 

Optimal solution found (tolerance 5.00e-02)
Best objective 9.5236863

Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.054549e+07, 17900 iterations, 0.82 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.0545e+07    0  816          - 1.0545e+07      -     -    4s
H    0     0                    1.143032e+07 1.0545e+07  7.74%     -    4s
H    0     0                    1.101858e+07 1.0545e+07  4.29%     -    4s

Explored 1 nodes (21937 simplex iterations) in 4.84 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 1.10186e+07 1.14303e+07 

Optimal solution found (tolerance 5.00e-02)
Best objective 1.101857819420e+07, best bound 1.054548644292e+07, gap 4.2936%
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\106.txt  Default: 
El programa 'Dimensionami


     0     0 9057305.28    0  556          - 9057305.28      -     -    3s
H    0     0                    9687424.1892 9057305.28  6.50%     -    3s
H    0     0                    9392821.7910 9057305.28  3.57%     -    3s

Explored 1 nodes (22096 simplex iterations) in 3.75 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 9.39282e+06 9.68742e+06 

Optimal solution found (tolerance 5.00e-02)
Best objective 9.392821791031e+06, best bound 9.057305275736e+06, gap 3.5721%
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\109.txt  Default: 
El programa 'Dimensionamiento de microrredes' es:  optimal
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                 : Upper
      1 :  None :   173.97424172564385 : 1500.0
      2 :  None : 0.017614299678369954 :    0.1
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                 : Upper
 

Thread count was 4 (of 4 available processors)

Solution count 2: 9.30588e+06 9.60976e+06 

Optimal solution found (tolerance 5.00e-02)
Best objective 9.305878829378e+06, best bound 8.980175549600e+06, gap 3.5000%
Changed value of parameter LogFile to 
   Prev: C:\Users\wrope\Documents\01. UdeA\00. Tesis\09. Implementacion\dimensionamiento_microrred\src\log_opt\112.txt  Default: 
El programa 'Dimensionamiento de microrredes' es:  optimal
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                 : Upper
      1 :  None :    174.3581477315512 : 1500.0
      2 :  None : 0.016794290828173554 :    0.1
max_ciclos_carga_descarga_lpsp : Size=2
    Key : Lower : Body                 : Upper
      1 :  None :    174.3581477315512 : 1500.0
      2 :  None : 0.016794290828173554 :    0.1
Terminado - Guardado de gráficos
simulacion 42 Terminada


 simulado: 42


#### paRAMETROS


#### FO Y RESTRICCIONES

In [7]:
#with pd.ExcelWriter('resultado_solver.xlsx') as writer:  
#
#    tabla_resultados.to_excel(writer, sheet_name='resultado')
#    
#os.startfile('resultado_solver.xlsx')

## Para guardar parametros

## Para Guardar en base de datos